In [ ]:
#Importing/mounting files from Google Drive

from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
#Unzipping file

!unzip /content/drive/MyDrive/mainfolder

Archive:  /content/drive/MyDrive/mainfolder.zip
   creating: mainfolder/
   creating: mainfolder/product_descriptions/
  inflating: mainfolder/product_descriptions/description_AISJDKFJW93NJ.txt  
  inflating: mainfolder/product_descriptions/description_DJKFIEI432FIE.txt  
  inflating: mainfolder/product_descriptions/description_GGOENEBJ079499.txt  
  inflating: mainfolder/product_descriptions/description_HJSKNWK429DJE.txt  
  inflating: mainfolder/product_descriptions/description_JFKL3940NFKLJ.txt  
  inflating: mainfolder/product_descriptions/description_LKDFJ49LSDJKL.txt  
  inflating: mainfolder/product_descriptions/description_MWKDI3JFK39SL.txt  
  inflating: mainfolder/product_descriptions/description_NEKFJOWE9FDIW.txt  
  inflating: mainfolder/product_descriptions/description_OWEJL398FWJLK.txt  
  inflating: mainfolder/product_descriptions/description_XPLFJW2490XJN.txt  
   creating: mainfolder/product_details/
  inflating: mainfolder/product_details/details_AISJDKFJW93NJ.json  


In [ ]:
# Importing files
import os
import csv
import json

# Defining main Path Folder
main_folder = "/content/drive/MyDrive/mainfolder"

# Defining load data and three dictionaries
def load_data(mainfolder=main_folder):
    sales_data = {}
    product_details = {}
    product_descriptions = {}

    # Reading CSV File
    csv_path = os.path.join(mainfolder, "sales_data.csv")
    if os.path.exists(csv_path):
        with open(csv_path, "r") as file:
            reader = csv.DictReader(file)
            for row in reader:
                sku = row["product_id"].strip()
                values = [int(row[col]) for col in row if col != "product_id"]
                sales_data[sku] = values

    # Reading JSON Files
    json_folder = os.path.join(mainfolder, "product_details")
    if os.path.exists(json_folder):
        for filename in os.listdir(json_folder):
            if filename.endswith(".json"):
                sku = filename.replace("details_", "").replace(".json", "")
                with open(os.path.join(json_folder, filename), "r") as f:
                    product_details[sku] = json.load(f)

    # Reading TXT Files
    txt_folder = os.path.join(mainfolder, "product_descriptions")
    if os.path.exists(txt_folder):
        for filename in os.listdir(txt_folder):
            if filename.endswith(".txt"):
                sku = filename.replace("description_", "").replace(".txt", "")
                with open(os.path.join(txt_folder, filename), "r") as f:
                    product_descriptions[sku] = f.read().strip()

    print("Data loaded successfully.")
    return product_details, sales_data, product_descriptions

# Creating functions for the admin to update
def update_sales_data(sales_dict, sku, new_sales):
    sales_dict[sku] = new_sales
    print(f"Sales data updated for {sku}")
    return sales_dict

def update_product_details(details_dict, sku, details):
    details_dict[sku] = details
    print(f"Product details updated for {sku}")
    return details_dict

def update_product_description(desc_dict, sku, description):
    desc_dict[sku] = description
    print(f"Product description updated for {sku}")
    return desc_dict

def update(product_details, sales_data, product_descriptions):
    sku = input("Enter Product SKU (13 characters): ").strip()
    if len(sku) != 13:
        print("Error: SKU must be exactly 13 characters.")
        return product_details, sales_data, product_descriptions

    sales_input = input("Enter last 14 days sales (14 whole numbers, space separated): ").strip().split()
    if len(sales_input) != 14 or not all(x.isdigit() for x in sales_input):
        print("Error: Enter exactly 14 whole numbers.")
        return product_details, sales_data, product_descriptions
    sales_list = [int(x) for x in sales_input]

    name = input("Enter Product Name: ").strip()
    brand = input("Enter Brand: ").strip()
    model = input("Enter Model: ").strip()
    specifications = input("Enter Specifications: ").strip()
    price = input("Enter Price (e.g. $1.99): ").strip()
    availability = input("Enter Availability: ").strip()

    description = input("Enter Product Description: ").strip()
    if description == "":
        print("Error: Product description cannot be empty.")
        return product_details, sales_data, product_descriptions

    details = {
        "Product Name": name,
        "Brand": brand,
        "Model": model,
        "Specifications": specifications,
        "Price": price,
        "Availability": availability
    }

    update_sales_data(sales_data, sku, sales_list)
    update_product_details(product_details, sku, details)
    update_product_description(product_descriptions, sku, description)

    print("Product data updated successfully.")
    return product_details, sales_data, product_descriptions

## Defining dump data
def dump_data(sales_data, product_details, product_descriptions, main_folder):
    os.makedirs(os.path.join(main_folder, "product_details"), exist_ok=True)
    os.makedirs(os.path.join(main_folder, "product_descriptions"), exist_ok=True)

     # Save CSV
    csv_path = os.path.join(main_folder, "sales_data.csv")
    with open(csv_path, "w", newline="") as file:
        fieldnames = ["product_id"] + [f"day_{i+1}" for i in range(14)]
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        for sku, sales_list in sales_data.items():
            fixed = (sales_list + [""] * 14)[:14]   # pad/cut to 14
            row = {"product_id": sku}
            for i, val in enumerate(fixed):
                row[f"day_{i+1}"] = val
            writer.writerow(row)

    # Save JSON
    for sku, details in product_details.items():
        json_path = os.path.join(main_folder, "product_details", f"details_{sku}.json")
        with open(json_path, "w") as f:
            json.dump(details, f, indent=4)

    # Save TXT
    for sku, desc in product_descriptions.items():
        txt_path = os.path.join(main_folder, "product_descriptions", f"description_{sku}.txt")
        with open(txt_path, "w") as f:
            f.write(desc)

    print("All data saved successfully!")

# For Final output
if __name__ == "__main__":
    # Load existing data
    product_details, sales_data, product_descriptions = load_data()

    # New product info
    sku = "CMWKCILOP27KF"
    new_sales = [8, 14, 16, 7, 15, 21, 14, 16, 32, 29, 26, 30, 25, 22]
    new_details = {
        "Product Name": "Pokemon Card",
        "Brand": "GameFreak",
        "Model": "ScarletViolet151",
        "Specifications": "Genuine, TCG, English",
        "Price": "$1.99",
        "Availability": "In stock"
    }
    new_description = "Original Pokemon TCG Pikachu card"

    print("\nUpdating data...")

    # Applying updates
    sales_data = update_sales_data(sales_data, sku, new_sales)
    product_details = update_product_details(product_details, sku, new_details)
    product_descriptions = update_product_description(product_descriptions, sku, new_description)

    # Display updated data
    print("\n-------------------------------")
    print(f"Updated Data for SKU: {sku}")
    print("-------------------------------")
    print(f"Sales Data: {sales_data[sku]}")
    print("\nProduct Details:")
    print(json.dumps(product_details[sku], indent=2))
    print("\nProduct Description:")
    print(product_descriptions[sku])
    print()

    # Save to disk
    dump_data(sales_data, product_details, product_descriptions, main_folder)

Data loaded successfully.

Updating data...
Sales data updated for CMWKCILOP27KF
Product details updated for CMWKCILOP27KF
Product description updated for CMWKCILOP27KF

-------------------------------
Updated Data for SKU: CMWKCILOP27KF
-------------------------------
Sales Data: [8, 14, 16, 7, 15, 21, 14, 16, 32, 29, 26, 30, 25, 22]

Product Details:
{
  "Product Name": "Pokemon Card",
  "Brand": "GameFreak",
  "Model": "ScarletViolet151",
  "Specifications": "Genuine, TCG, English",
  "Price": "$1.99",
  "Availability": "In stock"
}

Product Description:
Original Pokemon TCG Pikachu card

All data saved successfully!
